In [144]:
import warnings
warnings.filterwarnings("ignore")

In [145]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

from time import sleep

from dotenv import load_dotenv
load_dotenv()

True

In [146]:
email, pwd = os.environ['EMAIL'], os.environ['PASSWORD'] 

In [147]:
driver = webdriver.Chrome()

In [148]:
driver.get('https://www.linkedin.com/login')

In [149]:
driver.title

'LinkedIn Login, Sign in | LinkedIn'

In [150]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.environ['EMAIL'])

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ['PASSWORD'])

password.submit()

In [151]:
## MAKE SURE TO USE ONLY THIS URL TO AVOID BEING STUCK IN ERRORS

url = "https://uk.linkedin.com/in/austinfinnegan"
driver.get(url)

In [152]:
profile_data = {}

In [153]:
driver.title

'(14) Austin Finnegan | LinkedIn'

In [155]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

name_element = driver.find_element(By.XPATH, "//div[contains(@class, 'mt2')]//h1[contains(@class, 'inline t-24 v-align-middle break-words')]")

name = name_element.text.strip()

profile_data['name'] = name
profile_data['url'] = url

In [156]:
profile_data

{'name': 'Austin Finnegan', 'url': 'https://uk.linkedin.com/in/austinfinnegan'}

In [157]:
headline = soup.find('div', {'class': 'text-body-medium break-words'})
headline = headline.get_text().strip()

profile_data['headline'] = headline

In [158]:
profile_data

{'name': 'Austin Finnegan',
 'url': 'https://uk.linkedin.com/in/austinfinnegan',
 'headline': 'European Treasurer at Mercuria'}

### Read Experience

In [159]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

sections = soup.find_all('section', {'class': 'artdeco-card pv-profile-card break-words mt2'})

In [160]:
sections

[<section class="artdeco-card pv-profile-card break-words mt2" data-view-name="profile-card" tabindex="-1">
 <!-- -->
 <div class="pv-profile-card__anchor" id="highlights">
 </div>
 <!-- -->
 <!-- -->
 <div class="eXpkiawsnDKhbHXbkjHiWzcvMSXzCDTSbIF">
 <div class="pvs-header__top-container--no-stack">
 <div class="pvs-header__left-container--stack">
 <div class="aSfTsoDORSQyRuNSYoPvEOYZxkHuvSYDfxs">
 <!-- --> <h2 class="pvs-header__title text-heading-large">
 <span aria-hidden="true"><!-- -->L’essentiel<!-- --></span><span class="visually-hidden"><!-- -->L’essentiel<!-- --></span>
 </h2>
 <!-- --><!-- --> </div>
 <!-- --> </div>
 <!-- --> </div>
 </div>
 <div class="PCElbkussknghwOfRKfJMfjXDkKfBlMxpIAbEg">
 <!-- --> <ul class="LaWpEyxyxapNltWxwudDhdLxnPAtjyvJWU">
 <li class="artdeco-list__item XhFkfzTqVEjbrcAqjKxYJuWSTFQlKfmuurE unxmtDKBxJsHIqHhzyWRwitufIetjxGMMnwb">
 <!-- --> <div class="BfMYVkoegDeGnAjifroWtQCkcxUftzPwhOo HtHiZDWwtnuIZCnJNDWCcrhvzGCYFocrM atRtNSDCuBCKurXggRDUlbnAeEUE

In [161]:
import re

In [162]:
for sec in sections:
    if sec.find('div', {'id': 'experience'}):
        experience = sec

experience

<section class="artdeco-card pv-profile-card break-words mt2" data-view-name="profile-card" tabindex="-1">
<!-- -->
<div class="pv-profile-card__anchor" id="experience">
</div>
<!-- -->
<!-- -->
<div class="eXpkiawsnDKhbHXbkjHiWzcvMSXzCDTSbIF">
<div class="pvs-header__top-container--no-stack">
<div class="pvs-header__left-container--stack">
<div class="aSfTsoDORSQyRuNSYoPvEOYZxkHuvSYDfxs">
<!-- --> <h2 class="pvs-header__title text-heading-large">
<span aria-hidden="true"><!-- -->Expérience<!-- --></span><span class="visually-hidden"><!-- -->Expérience<!-- --></span>
</h2>
<!-- --><!-- --> </div>
<!-- --> </div>
<!-- --> </div>
</div>
<div class="PCElbkussknghwOfRKfJMfjXDkKfBlMxpIAbEg">
<!-- --> <ul class="LaWpEyxyxapNltWxwudDhdLxnPAtjyvJWU">
<li class="artdeco-list__item XhFkfzTqVEjbrcAqjKxYJuWSTFQlKfmuurE unxmtDKBxJsHIqHhzyWRwitufIetjxGMMnwb">
<!-- --> <div class="BfMYVkoegDeGnAjifroWtQCkcxUftzPwhOo HtHiZDWwtnuIZCnJNDWCcrhvzGCYFocrM atRtNSDCuBCKurXggRDUlbnAeEUEjvrdktJYs" data-view-na

In [163]:
for section in sections:
    heading = section.find('h2')
    if heading and ("Expérience" in heading.text or "Experience" in heading.text):  
        experience_section = section
        break  

if experience_section:
    experience_entries = experience_section.find_all('li', {'class': 'artdeco-list__item'})
else:
    experience_entries = []

experience_data = []

for entry in experience_entries:
    try:
        job_title_elem = entry.find('div', {'class': 'display-flex align-items-center mr1 hoverable-link-text t-bold'})
        job_title = job_title_elem.get_text(strip=True) if job_title_elem else "N/A"

        company_name_elem = entry.find('span', {'class': 't-normal'})
        company_name = company_name_elem.get_text(strip=True) if company_name_elem else "N/A"

        duration_elem = entry.find('span', {'class': 't-black--light'})
        work_duration = duration_elem.get_text(strip=True) if duration_elem else "N/A"

        location_elem = duration_elem.find_next('span', {'class': 't-black--light'}) if duration_elem else None
        location = location_elem.get_text(strip=True) if location_elem else "N/A"

        description_elem = entry.find('div', {'class': 'inline-show-more-text'})
        job_description = description_elem.get_text(strip=True) if description_elem else "N/A"

        experience_data.append({
            "Job Title": job_title,
            "Company": company_name,
            "Duration": work_duration,
            "Location": location,
            "Description": job_description
        })
    except AttributeError:
        continue

df_experience = pd.DataFrame(experience_data)

In [164]:
df_experience

,Job Title,Company,Duration,Location,Description
0,MercuriaMercuria,Temps plein · 6 ans 7 moisTemps plein · 6 ans ...,déc. 2024 - aujourd’hui · 6 moisDe déc. 2024 à...,"London, United KingdomLondon, United Kingdom",N/A
1,Board of GovernorsBoard of Governors,St Nicholas CE Primary School · Temps partielS...,oct. 2023 - aujourd’hui · 1 an 8 moisDe oct. 2...,"Harpenden, England, United KingdomHarpenden, E...",N/A
2,Goldman SachsGoldman Sachs,8 ans 5 mois8 ans 5 mois,"London, United KingdomLondon, United Kingdom",août 2015 - nov. 2018 · 3 ans 4 moisDu août 20...,N/A
3,Sterling Markets Division (Research Analyst)St...,Bank of EnglandBank of England,juin 2008 - juil. 2009 · 1 an 2 moisDu juin 20...,"London, United KingdomLondon, United Kingdom",N/A


In [ ]:
def get_exp(exp):

    exp_dict = {}

    name = exp.find('div', {'class': 'display-flex flex-wrap align-items-center full-height'})
    name = name.find('span', {'class': 'visually-hidden'})
    name = name.get_text().strip()

    duration = exp.find('span', {'class': 't-14 t-normal'})
    duration = duration.find('span', {'class': 'visually-hidden'})
    duration = duration.get_text().strip()
    duration

    exp_dict['company_name'] = name
    exp_dict['duration'] = duration

    designations = exp.find_all('div', {'class': 'gFJNglFOnyZmIAbxVkrWpQCmMhGSasZRfRtGlFg'})

    item_list = []
    for position in designations:
        spans = position.find_all('span', {'class': 'visually-hidden'})

        item_dict = {}
        item_dict['designation'] = spans[0].get_text().strip()
        item_dict['duration'] = spans[1].get_text().strip()
        item_dict['location'] = spans[2].get_text().strip()

        try:
            item_dict['projects'] = spans[3].get_text().strip()
        except:
            item_dict['projects'] = ""

        item_list.append(item_dict)


    exp_dict['designations'] = item_list

    return exp_dict

item_list = []
for exp in experience:
    item_list.append(get_exp(exp))

AttributeError: 'NoneType' object has no attribute 'find'

: 

In [61]:
profile_data['experience'] = item_list


In [63]:
# profile_data

### Education

In [64]:
for sec in sections:
    if sec.find('div', {'id': 'education'}):
        educations = sec

In [69]:
# educations.get_text().strip()

In [70]:
items = educations.find_all('div', {'class': 'gFJNglFOnyZmIAbxVkrWpQCmMhGSasZRfRtGlFg YcvguSXGGYgEmpWJOhQrhmNocNiIvvDjETE bAMoJMoZyFCvvmHkHNQEBnMuJTWUocrss'})

In [72]:
len(items)

2

In [77]:
def get_edu(item):
    item_dict = {}
    spans = item.find_all('span', {'class': 'visually-hidden'})

    item_dict['college'] = spans[0].get_text().strip()
    item_dict['degree'] = spans[1].get_text().strip()
    item_dict['duration'] = spans[2].get_text().strip()
    item_dict['project'] = spans[3].get_text().strip()

    return item_dict

item_list = []
for item in items:
    item_list.append(get_edu(item))

profile_data['education'] = item_list


In [80]:
# profile_data

### Licenses & certifications

In [81]:
driver.find_element(By.ID, "navigation-index-see-all-licenses-and-certifications").click()

In [83]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

soup = soup.find('section', {'class': 'artdeco-card pb3'})

In [84]:
items = soup.find_all('div', {'class': 'gFJNglFOnyZmIAbxVkrWpQCmMhGSasZRfRtGlFg YcvguSXGGYgEmpWJOhQrhmNocNiIvvDjETE bAMoJMoZyFCvvmHkHNQEBnMuJTWUocrss'})

len(items)

6

In [87]:
item = items[0]

def get_license(item):
    spans = item.find_all('span', {'class': 'visually-hidden'})

    item_dict = {}
    item_dict['name'] = spans[0].get_text().strip()
    item_dict['institute'] = spans[1].get_text().strip()
    item_dict['issued_date'] = spans[2].get_text().strip()

    return item_dict

item_list = []
for item in items:
    item_list.append(get_license(item))

profile_data['licenses'] = item_list


In [90]:
driver.back()
# profile_data


### Projects

In [91]:
driver.find_element(By.ID, "navigation-index-see-all-projects").click()

In [92]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

soup = soup.find('section', {'class': 'artdeco-card pb3'})

In [95]:
items = soup.find_all('div', {'class': 'gFJNglFOnyZmIAbxVkrWpQCmMhGSasZRfRtGlFg YcvguSXGGYgEmpWJOhQrhmNocNiIvvDjETE bAMoJMoZyFCvvmHkHNQEBnMuJTWUocrss'})

len(items)

5

In [98]:
item = items[0]

def get_project(item):
    spans = item.find_all('span', {'class': 'visually-hidden'})

    item_dict = {}
    item_dict['project_name'] = spans[0].get_text().strip()
    item_dict['duration'] = spans[1].get_text().strip()
    item_dict['description'] = spans[2].get_text().strip()

    return item_dict

item_list = []
for item in items:
    item_list.append(get_project(item))

profile_data['projects'] = item_list

In [100]:
driver.back()

### All Courses

In [101]:
driver.find_element(By.ID, "navigation-index-see-all-courses").click()

In [102]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

soup = soup.find('section', {'class': 'artdeco-card pb3'})

items = soup.find_all('div', {'class': 'gFJNglFOnyZmIAbxVkrWpQCmMhGSasZRfRtGlFg YcvguSXGGYgEmpWJOhQrhmNocNiIvvDjETE bAMoJMoZyFCvvmHkHNQEBnMuJTWUocrss'})

len(items)

18

In [105]:
item = items[0]

def get_course(item):
    spans = item.find_all('span', {'class': 'visually-hidden'})

    item_dict = {}
    item_dict['course_name'] = spans[0].get_text().strip()
    try:
        item_dict['associated_with'] = spans[1].get_text().strip()
    except:
        item_dict['associated_with'] = ""

    return item_dict

item_list = []
for item in items:
    item_list.append(get_course(item))

profile_data['courses'] = item_list

In [107]:
driver.back()

### Honors & awards

In [108]:
driver.find_element(By.ID, "navigation-index-see-all-honorsandawards").click()

In [111]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

soup = soup.find('section', {'class': 'artdeco-card pb3'})

items = soup.find_all('span', {'class': 'visually-hidden'})

len(items)

6

In [113]:
item_list = []
for item in items:
    item_list.append(item.get_text().strip())

In [115]:
profile_data['honors_and_awards'] = item_list

In [116]:
import json

with open('data/profile_data_tutorial.json', 'w') as f:
    json.dump(profile_data, f, indent=4)

In [117]:
driver.back()